In [1]:
import numpy
from sklearn.feature_extraction import DictVectorizer 
import h2o 
from h2o.estimators.random_forest import H2ORandomForestEstimator 
import pandas 
from function import * 
pandas.options.display.max_rows=1000 

In [2]:
h2o.init()

Connecting to H2O server at http://localhost:54321... successful!


H2O cluster uptime:,17 hours 57 mins
H2O cluster version:,3.10.0.3
H2O cluster version age:,24 days
H2O cluster name:,H2O_from_python_I_1kgl2b
H2O cluster total nodes:,1
H2O cluster free memory:,3.157 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster is healthy:,True
H2O cluster is locked:,True
H2O connection url:,http://localhost:54321


In [3]:
collection = mongo_collection() 
records = collection.find({'$or': [{'type': 'apartments'}, {'type': 'hotel'}]})

In [4]:
train_records, valid_records = split_records(records, 0.8)

In [5]:
train_features, train_y = rooms_records(train_records) 
valid_features, valid_y = rooms_records(valid_records) 

In [6]:
train_vectorizer = DictVectorizer() 
X_train = train_vectorizer.fit_transform(train_features)

In [7]:
train = pandas.DataFrame(X_train.toarray(), columns=normalize_features(train_vectorizer.get_feature_names())) 
train['type'] = train_y  
X_valid = train_vectorizer.transform(valid_features)
valid = pandas.DataFrame(X_valid.toarray(), columns=normalize_features(train_vectorizer.get_feature_names())) 
valid['type'] = valid_y  

In [8]:
train_water = h2o.H2OFrame.from_python(train.to_dict('list')) 
valid_water = h2o.H2OFrame.from_python(valid.to_dict('list'))

#Random Forest Model with Size feature 

In [9]:
rf_size = H2ORandomForestEstimator(ntrees=100, max_depth=25, min_rows=5)

In [10]:
xc = train_water.columns.copy() 
xc.remove('type')  

In [11]:
rf_size.train(training_frame=train_water, validation_frame=valid_water, x=xc, y='type')

In [12]:
rf_size

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  DRF_model_python_1472160914002_741

Model Summary: 


,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,100.0,100.0,6091919.0,25.0,25.0,25.0,4506.0,5217.0,4803.21




ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.04415260474500531
RMSE: 0.2101252120641531
LogLoss: 0.15288865878418667
Mean Per-Class Error: 0.0784998704365446
AUC: 0.9780828552357246
Gini: 0.9561657104714492

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4710242113071299: 


,apartments,hotel,Error,Rate
apartments,31392.0,7561.0,0.1941,(7561.0/38953.0)
hotel,3964.0,166324.0,0.0233,(3964.0/170288.0)
Total,35356.0,173885.0,0.0551,(11525.0/209241.0)



Maximum Metrics: Maximum metrics at their respective thresholds



...



Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_lift,validation_classification_error
,2016-08-26 08:39:35,0.005 sec,0.0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
,2016-08-26 08:39:36,1.469 sec,1.0,0.2466771,0.6322210,0.9442548,1.2227071,0.0812620,0.2710010,1.0154181,0.9167781,1.1995524,0.0940301
,2016-08-26 08:39:38,2.936 sec,2.0,0.2424982,0.5399984,0.9490407,1.2281566,0.0778270,0.2500381,0.4578837,0.9450643,1.2158459,0.0837212
,2016-08-26 08:39:44,9.276 sec,7.0,0.2272570,0.2863542,0.9643934,1.2287478,0.0688601,0.2344919,0.2167320,0.9618390,1.2218235,0.0729123
,2016-08-26 08:39:59,23.770 sec,20.0,0.2148414,0.1644771,0.9751289,1.2287478,0.0598248,0.2308317,0.1846481,0.9658827,1.2134629,0.0693926
,2016-08-26 08:40:16,41.525 sec,36.0,0.2120849,0.1554237,0.9769583,1.2287478,0.0567862,0.2299544,0.1803526,0.9669445,1.2253754,0.0689503
,2016-08-26 08:40:45,1 min 10.047 sec,61.0,0.2108954,0.1536408,0.9776101,1.2287478,0.0556918,0.2296926,0.1799003,0.9672669,1.2250712,0.0687579
,2016-08-26 08:41:27,1 min 52.012 sec,99.0,0.2101391,0.1529142,0.9780747,1.2287478,0.0550322,0.2295380,0.1784641,0.9676501,1.2295505,0.0685656
,2016-08-26 08:41:34,1 min 59.296 sec,100.0,0.2101252,0.1528887,0.9780829,1.2287478,0.0550800,0.2295423,0.1784733,0.9676362,1.2295505,0.0685848



Variable Importances: 


variable,relative_importance,scaled_importance,percentage
_Kitchenware,221704.7343750,1.0,0.1338933
_Stovetop,209104.2031250,0.9431653,0.1262836
_Dining_area,113938.7578125,0.5139212,0.0688106
_Kitchen,95666.7421875,0.4315052,0.0577757
_Oven,82197.3281250,0.3707513,0.0496411
---,---,---,---
_Game_console___Nintendo_Wii,7.1290483,0.0000322,0.0000043
_Fold_up_bed,3.3647594,0.0000152,0.0000020
_Game_console___PS3,3.1176538,0.0000141,0.0000019
_Game_console___Xbox_360,1.1429421,0.0000052,0.0000007



See the whole table with table.as_data_frame()


In [13]:
rf_size.model_performance()


ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.04415260474500531
RMSE: 0.2101252120641531
LogLoss: 0.15288865878418667
Mean Per-Class Error: 0.0784998704365446
AUC: 0.9780828552357246
Gini: 0.9561657104714492

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4710242113071299: 


,apartments,hotel,Error,Rate
apartments,31392.0,7561.0,0.1941,(7561.0/38953.0)
hotel,3964.0,166324.0,0.0233,(3964.0/170288.0)
Total,35356.0,173885.0,0.0551,(11525.0/209241.0)



Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.4710242,0.9665139,231.0
max f2,0.3392903,0.9779084,272.0
max f0point5,0.7265174,0.9682858,147.0
max accuracy,0.4742926,0.9449343,230.0
max precision,0.9999155,0.9999395,0.0
max recall,0.0089489,1.0,396.0
max specificity,0.9999155,0.9999487,0.0
max absolute_mcc,0.5127576,0.8142324,218.0
max min_per_class_accuracy,0.7265174,0.9207813,147.0
max mean_per_class_accuracy,0.7385904,0.9215001,143.0



Gains/Lift Table: Avg response rate: 81.38 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0100172,0.9999748,1.2287478,1.2287478,1.0,1.0,0.0123086,0.0123086,22.8747768,22.8747768
,2,0.0200104,0.9999600,1.2287478,1.2287478,1.0,1.0,0.0122792,0.0245878,22.8747768,22.8747768
,3,0.0300037,0.9999472,1.2287478,1.2287478,1.0,1.0,0.0122792,0.0368670,22.8747768,22.8747768
,4,0.0400017,0.9999351,1.2287478,1.2287478,1.0,1.0,0.0122851,0.0491520,22.8747768,22.8747768
,5,0.0730689,0.9999247,1.2287478,1.2287478,1.0,1.0,0.0406312,0.0897832,22.8747768,22.8747768
,6,0.1000091,0.9998681,1.2287478,1.2287478,1.0,1.0,0.0331027,0.1228859,22.8747768,22.8747768
,7,0.1500041,0.9996668,1.2287478,1.2287478,1.0,1.0,0.0614312,0.1843172,22.8747768,22.8747768
,8,0.2000086,0.9991216,1.2287478,1.2287478,1.0,1.0,0.0614430,0.2457601,22.8747768,22.8747768
,9,0.3000033,0.9965742,1.2287478,1.2287478,1.0,1.0,0.1228683,0.3686284,22.8747768,22.8747768
,10,0.4000220,0.9911660,1.2286891,1.2287331,0.9999522,0.9999881,0.1228918,0.4915202,22.8689055,22.8733088


In [14]:
rf_size.show()

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  DRF_model_python_1472160914002_741

Model Summary: 


,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,100.0,100.0,6091919.0,25.0,25.0,25.0,4506.0,5217.0,4803.21




ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.04415260474500531
RMSE: 0.2101252120641531
LogLoss: 0.15288865878418667
Mean Per-Class Error: 0.0784998704365446
AUC: 0.9780828552357246
Gini: 0.9561657104714492

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4710242113071299: 


,apartments,hotel,Error,Rate
apartments,31392.0,7561.0,0.1941,(7561.0/38953.0)
hotel,3964.0,166324.0,0.0233,(3964.0/170288.0)
Total,35356.0,173885.0,0.0551,(11525.0/209241.0)



Maximum Metrics: Maximum metrics at their respective thresholds



...



Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_lift,validation_classification_error
,2016-08-26 08:39:35,0.005 sec,0.0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
,2016-08-26 08:39:36,1.469 sec,1.0,0.2466771,0.6322210,0.9442548,1.2227071,0.0812620,0.2710010,1.0154181,0.9167781,1.1995524,0.0940301
,2016-08-26 08:39:38,2.936 sec,2.0,0.2424982,0.5399984,0.9490407,1.2281566,0.0778270,0.2500381,0.4578837,0.9450643,1.2158459,0.0837212
,2016-08-26 08:39:44,9.276 sec,7.0,0.2272570,0.2863542,0.9643934,1.2287478,0.0688601,0.2344919,0.2167320,0.9618390,1.2218235,0.0729123
,2016-08-26 08:39:59,23.770 sec,20.0,0.2148414,0.1644771,0.9751289,1.2287478,0.0598248,0.2308317,0.1846481,0.9658827,1.2134629,0.0693926
,2016-08-26 08:40:16,41.525 sec,36.0,0.2120849,0.1554237,0.9769583,1.2287478,0.0567862,0.2299544,0.1803526,0.9669445,1.2253754,0.0689503
,2016-08-26 08:40:45,1 min 10.047 sec,61.0,0.2108954,0.1536408,0.9776101,1.2287478,0.0556918,0.2296926,0.1799003,0.9672669,1.2250712,0.0687579
,2016-08-26 08:41:27,1 min 52.012 sec,99.0,0.2101391,0.1529142,0.9780747,1.2287478,0.0550322,0.2295380,0.1784641,0.9676501,1.2295505,0.0685656
,2016-08-26 08:41:34,1 min 59.296 sec,100.0,0.2101252,0.1528887,0.9780829,1.2287478,0.0550800,0.2295423,0.1784733,0.9676362,1.2295505,0.0685848



Variable Importances: 


variable,relative_importance,scaled_importance,percentage
_Kitchenware,221704.7343750,1.0,0.1338933
_Stovetop,209104.2031250,0.9431653,0.1262836
_Dining_area,113938.7578125,0.5139212,0.0688106
_Kitchen,95666.7421875,0.4315052,0.0577757
_Oven,82197.3281250,0.3707513,0.0496411
---,---,---,---
_Game_console___Nintendo_Wii,7.1290483,0.0000322,0.0000043
_Fold_up_bed,3.3647594,0.0000152,0.0000020
_Game_console___PS3,3.1176538,0.0000141,0.0000019
_Game_console___Xbox_360,1.1429421,0.0000052,0.0000007



See the whole table with table.as_data_frame()


In [15]:
rf_size.confusion_matrix(valid=True, metrics = ['f1', 'f2', 'absolute_mcc', 'accuracy', 'precision', 'f0point5', 'min_per_class_accuracy', 'mean_per_class_accuracy'])


Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4669421180264029: 


,apartments,hotel,Error,Rate
apartments,7501.0,2206.0,0.2273,(2206.0/9707.0)
hotel,1360.0,40927.0,0.0322,(1360.0/42287.0)
Total,8861.0,43133.0,0.0686,(3566.0/51994.0)



Confusion Matrix (Act/Pred) for max f2 @ threshold = 0.2823098875554623: 


,apartments,hotel,Error,Rate
apartments,5640.0,4067.0,0.419,(4067.0/9707.0)
hotel,453.0,41834.0,0.0107,(453.0/42287.0)
Total,6093.0,45901.0,0.0869,(4520.0/51994.0)



Confusion Matrix (Act/Pred) for max absolute_mcc @ threshold = 0.5337082844909848: 


...


Confusion Matrix (Act/Pred) for max min_per_class_accuracy @ threshold = 0.7266489198985264: 


,apartments,hotel,Error,Rate
apartments,8803.0,904.0,0.0931,(904.0/9707.0)
hotel,3984.0,38303.0,0.0942,(3984.0/42287.0)
Total,12787.0,39207.0,0.094,(4888.0/51994.0)



Confusion Matrix (Act/Pred) for max mean_per_class_accuracy @ threshold = 0.720860887617231: 


,apartments,hotel,Error,Rate
apartments,8785.0,922.0,0.095,(922.0/9707.0)
hotel,3892.0,38395.0,0.092,(3892.0/42287.0)
Total,12677.0,39317.0,0.0926,(4814.0/51994.0)


[, , , , , , , ]

In [16]:
rf_size.varimp(use_pandas=True)

,variable,relative_importance,scaled_importance,percentage
0,_Kitchenware,221704.734375,1.000000,1.338933e-01
1,_Stovetop,209104.203125,0.943165,1.262836e-01
2,_Dining_area,113938.757812,0.513921,6.881063e-02
3,_Kitchen,95666.742188,0.431505,5.777567e-02
4,_Oven,82197.328125,0.370751,4.964114e-02
5,_Washing_Machine,75558.843750,0.340808,4.563198e-02
6,_Size,66889.226562,0.301704,4.039617e-02
7,_Private_flat_in_building,52155.703125,0.235248,3.149821e-02
8,_Kitchenette,46345.996094,0.209044,2.798957e-02
9,_Toaster,45586.625000,0.205619,2.753097e-02


In [17]:
xc.remove('_Size')  

In [18]:
xc

['_View_Lake',
 '_Executive_Lounge_Access',
 '_Telephone',
 '_Board_games_puzzles',
 '_Wardrobe_Closet',
 '_Toilet',
 '_Detached',
 '_Shared_Toilet',
 '_Desk',
 '_Balcony',
 '_Private_flat_in_building',
 '_Flat_screen_TV',
 '_Iron',
 '_Alarm_clock',
 '_Satellite_Channels',
 '_Game_console___Xbox_360',
 '_Toaster',
 '_Air_Conditioning',
 '_Fan',
 '_Spa_Bath',
 '_Free_toiletries',
 '_Towels_Sheets_extra_fee',
 '_View_Pool',
 '_View_City',
 '_Additional_Toilet',
 '_Blu_ray_player',
 '_Hypoallergenic',
 '_Entire_unit_wheelchair_accessible',
 '_Game_console___PS2',
 '_TV',
 '_Game_console___Nintendo_Wii',
 '_Kitchenware',
 '_Additional_bathroom',
 '_Trouser_Press',
 '_Cleaning_products',
 '_Slippers',
 '_Extra_Long_Beds_greater_2_metres',
 '_Towels',
 '_Shared_Bathroom',
 '_Laptop_safe',
 '_Child_safety_socket_covers',
 '_DVD_Player',
 '_Electric_kettle',
 '_Video',
 '_Coffee_machine',
 '_Game_console___PS3',
 '_Dishwasher',
 '_Barbecue',
 '_View_River',
 '_Linen',
 '_Bidet',
 '_Refrigerato

#Random Forest Model without Size feature 

In [19]:
rf = H2ORandomForestEstimator(ntrees=100, max_depth=25, min_rows=5, col_sample_rate_change_per_level=1.65) 

In [20]:
rf.train(training_frame=train_water, validation_frame=valid_water, x=xc, y='type') 

In [21]:
rf  

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  DRF_model_python_1472160914002_777

Model Summary: 


,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,100.0,100.0,5138156.0,25.0,25.0,25.0,3881.0,4310.0,4057.33




ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.03929004542299342
RMSE: 0.19821716732663047
LogLoss: 0.1397803247006909
Mean Per-Class Error: 0.07132226163588384
AUC: 0.9807863100612783
Gini: 0.9615726201225565

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.46675616652092905: 


,apartments,hotel,Error,Rate
apartments,31967.0,6986.0,0.1793,(6986.0/38953.0)
hotel,3329.0,166959.0,0.0195,(3329.0/170288.0)
Total,35296.0,173945.0,0.0493,(10315.0/209241.0)



Maximum Metrics: Maximum metrics at their respective thresholds



...



Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_lift,validation_classification_error
,2016-08-26 08:41:41,0.003 sec,0.0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
,2016-08-26 08:41:43,2.060 sec,1.0,0.2380614,0.7913803,0.9442182,1.2216314,0.0722556,0.2758131,1.3790175,0.9038336,1.1904110,0.0934916
,2016-08-26 08:41:49,7.702 sec,4.0,0.2278007,0.5456034,0.9541888,1.2287033,0.0675404,0.2441672,0.3811527,0.9490060,1.2163200,0.0794707
,2016-08-26 08:41:56,15.158 sec,8.0,0.2161015,0.3253927,0.9667835,1.2287478,0.0612278,0.2373760,0.2570336,0.9574194,1.2153888,0.0754510
,2016-08-26 08:42:06,24.948 sec,13.0,0.2081927,0.2166525,0.9741153,1.2287478,0.0560671,0.2336820,0.2205360,0.9608687,1.2187022,0.0712775
,2016-08-26 08:42:20,38.717 sec,20.0,0.2036595,0.1712976,0.9776641,1.2287478,0.0531835,0.2321886,0.2044098,0.9626153,1.2207312,0.0704889
,2016-08-26 08:42:39,58.099 sec,30.0,0.2012078,0.1537242,0.9791415,1.2287478,0.0511563,0.2310782,0.1962317,0.9635263,1.2204785,0.0698157
,2016-08-26 08:43:03,1 min 21.453 sec,42.0,0.1999943,0.1468768,0.9798788,1.2287478,0.0505159,0.2307181,0.1889193,0.9642866,1.2244989,0.0692772
,2016-08-26 08:43:32,1 min 51.263 sec,57.0,0.1991236,0.1424680,0.9803453,1.2287478,0.0498134,0.2302380,0.1861286,0.9647335,1.2229186,0.0689310
,2016-08-26 08:44:14,2 min 32.750 sec,77.0,0.1986700,0.1410656,0.9805874,1.2287478,0.0495649,0.2301293,0.1845329,0.9648168,1.2238316,0.0691618



Variable Importances: 


variable,relative_importance,scaled_importance,percentage
_Kitchenware,253839.2968750,1.0,0.1423139
_Stovetop,168857.8281250,0.6652155,0.0946694
_Oven,129406.4140625,0.5097966,0.0725512
_Kitchen,84596.8203125,0.3332692,0.0474288
_Washing_Machine,82866.3359375,0.3264520,0.0464587
---,---,---,---
_iPad,2.6219268,0.0000103,0.0000015
_DVDs_or_music_for_children,2.3053477,0.0000091,0.0000013
_Game_console___PS3,1.8623310,0.0000073,0.0000010
_Game_console___Xbox_360,0.0,0.0,0.0



See the whole table with table.as_data_frame()


In [22]:
rf.model_performance() 


ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.03929004542299342
RMSE: 0.19821716732663047
LogLoss: 0.1397803247006909
Mean Per-Class Error: 0.07132226163588384
AUC: 0.9807863100612783
Gini: 0.9615726201225565

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.46675616652092905: 


,apartments,hotel,Error,Rate
apartments,31967.0,6986.0,0.1793,(6986.0/38953.0)
hotel,3329.0,166959.0,0.0195,(3329.0/170288.0)
Total,35296.0,173945.0,0.0493,(10315.0/209241.0)



Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.4667562,0.9700348,229.0
max f2,0.3346077,0.9800841,267.0
max f0point5,0.7222422,0.9713450,148.0
max accuracy,0.4995731,0.9507792,219.0
max precision,0.9996489,0.9996012,1.0
max recall,0.0000680,1.0,399.0
max specificity,0.9999427,0.9994095,0.0
max absolute_mcc,0.5030803,0.8341076,218.0
max min_per_class_accuracy,0.7207179,0.9271173,149.0
max mean_per_class_accuracy,0.7243620,0.9286777,147.0



Gains/Lift Table: Avg response rate: 81.38 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0122299,1.0,1.2287478,1.2287478,1.0,1.0,0.0150275,0.0150275,22.8747768,22.8747768
,2,0.0201872,0.9999930,1.2287478,1.2287478,1.0,1.0,0.0097776,0.0248050,22.8747768,22.8747768
,3,0.0336406,0.9999846,1.2287478,1.2287478,1.0,1.0,0.0165308,0.0413359,22.8747768,22.8747768
,4,0.0414737,0.9999836,1.2287478,1.2287478,1.0,1.0,0.0096249,0.0509607,22.8747768,22.8747768
,5,0.0500237,0.9999808,1.2287478,1.2287478,1.0,1.0,0.0105057,0.0614665,22.8747768,22.8747768
,6,0.1002672,0.9999664,1.2287478,1.2287478,1.0,1.0,0.0617366,0.1232030,22.8747768,22.8747768
,7,0.1500232,0.9999367,1.2287478,1.2287478,1.0,1.0,0.0611376,0.1843406,22.8747768,22.8747768
,8,0.2008545,0.9999083,1.2287478,1.2287478,1.0,1.0,0.0624589,0.2467995,22.8747768,22.8747768
,9,0.3000177,0.9995149,1.2287478,1.2287478,1.0,1.0,0.1218465,0.3686461,22.8747768,22.8747768
,10,0.4000315,0.9974603,1.2287478,1.2287478,1.0,1.0,0.1228918,0.4915379,22.8747768,22.8747768


In [23]:
rf.show() 

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  DRF_model_python_1472160914002_777

Model Summary: 


,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,100.0,100.0,5138156.0,25.0,25.0,25.0,3881.0,4310.0,4057.33




ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.03929004542299342
RMSE: 0.19821716732663047
LogLoss: 0.1397803247006909
Mean Per-Class Error: 0.07132226163588384
AUC: 0.9807863100612783
Gini: 0.9615726201225565

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.46675616652092905: 


,apartments,hotel,Error,Rate
apartments,31967.0,6986.0,0.1793,(6986.0/38953.0)
hotel,3329.0,166959.0,0.0195,(3329.0/170288.0)
Total,35296.0,173945.0,0.0493,(10315.0/209241.0)



Maximum Metrics: Maximum metrics at their respective thresholds



...



Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_lift,validation_classification_error
,2016-08-26 08:41:41,0.003 sec,0.0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
,2016-08-26 08:41:43,2.060 sec,1.0,0.2380614,0.7913803,0.9442182,1.2216314,0.0722556,0.2758131,1.3790175,0.9038336,1.1904110,0.0934916
,2016-08-26 08:41:49,7.702 sec,4.0,0.2278007,0.5456034,0.9541888,1.2287033,0.0675404,0.2441672,0.3811527,0.9490060,1.2163200,0.0794707
,2016-08-26 08:41:56,15.158 sec,8.0,0.2161015,0.3253927,0.9667835,1.2287478,0.0612278,0.2373760,0.2570336,0.9574194,1.2153888,0.0754510
,2016-08-26 08:42:06,24.948 sec,13.0,0.2081927,0.2166525,0.9741153,1.2287478,0.0560671,0.2336820,0.2205360,0.9608687,1.2187022,0.0712775
,2016-08-26 08:42:20,38.717 sec,20.0,0.2036595,0.1712976,0.9776641,1.2287478,0.0531835,0.2321886,0.2044098,0.9626153,1.2207312,0.0704889
,2016-08-26 08:42:39,58.099 sec,30.0,0.2012078,0.1537242,0.9791415,1.2287478,0.0511563,0.2310782,0.1962317,0.9635263,1.2204785,0.0698157
,2016-08-26 08:43:03,1 min 21.453 sec,42.0,0.1999943,0.1468768,0.9798788,1.2287478,0.0505159,0.2307181,0.1889193,0.9642866,1.2244989,0.0692772
,2016-08-26 08:43:32,1 min 51.263 sec,57.0,0.1991236,0.1424680,0.9803453,1.2287478,0.0498134,0.2302380,0.1861286,0.9647335,1.2229186,0.0689310
,2016-08-26 08:44:14,2 min 32.750 sec,77.0,0.1986700,0.1410656,0.9805874,1.2287478,0.0495649,0.2301293,0.1845329,0.9648168,1.2238316,0.0691618



Variable Importances: 


variable,relative_importance,scaled_importance,percentage
_Kitchenware,253839.2968750,1.0,0.1423139
_Stovetop,168857.8281250,0.6652155,0.0946694
_Oven,129406.4140625,0.5097966,0.0725512
_Kitchen,84596.8203125,0.3332692,0.0474288
_Washing_Machine,82866.3359375,0.3264520,0.0464587
---,---,---,---
_iPad,2.6219268,0.0000103,0.0000015
_DVDs_or_music_for_children,2.3053477,0.0000091,0.0000013
_Game_console___PS3,1.8623310,0.0000073,0.0000010
_Game_console___Xbox_360,0.0,0.0,0.0



See the whole table with table.as_data_frame()


In [24]:
rf.confusion_matrix(valid=True, metrics = ['f1', 'f2', 'absolute_mcc', 'accuracy', 'precision', 'f0point5', 'min_per_class_accuracy', 'mean_per_class_accuracy'])


Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.47608573762556683: 


,apartments,hotel,Error,Rate
apartments,7489.0,2218.0,0.2285,(2218.0/9707.0)
hotel,1362.0,40925.0,0.0322,(1362.0/42287.0)
Total,8851.0,43143.0,0.0689,(3580.0/51994.0)



Confusion Matrix (Act/Pred) for max f2 @ threshold = 0.2700727339195354: 


,apartments,hotel,Error,Rate
apartments,5779.0,3928.0,0.4047,(3928.0/9707.0)
hotel,542.0,41745.0,0.0128,(542.0/42287.0)
Total,6321.0,45673.0,0.086,(4470.0/51994.0)



Confusion Matrix (Act/Pred) for max absolute_mcc @ threshold = 0.49069249519392055: 


...


Confusion Matrix (Act/Pred) for max min_per_class_accuracy @ threshold = 0.7270376455240384: 


,apartments,hotel,Error,Rate
apartments,8781.0,926.0,0.0954,(926.0/9707.0)
hotel,4026.0,38261.0,0.0952,(4026.0/42287.0)
Total,12807.0,39187.0,0.0952,(4952.0/51994.0)



Confusion Matrix (Act/Pred) for max mean_per_class_accuracy @ threshold = 0.7519596906351474: 


,apartments,hotel,Error,Rate
apartments,8876.0,831.0,0.0856,(831.0/9707.0)
hotel,4399.0,37888.0,0.104,(4399.0/42287.0)
Total,13275.0,38719.0,0.1006,(5230.0/51994.0)


[, , , , , , , ]

In [25]:
rf.varimp(use_pandas=True) 

,variable,relative_importance,scaled_importance,percentage
0,_Kitchenware,253839.296875,1.000000,0.142314
1,_Stovetop,168857.828125,0.665215,0.094669
2,_Oven,129406.414062,0.509797,0.072551
3,_Kitchen,84596.820312,0.333269,0.047429
4,_Washing_Machine,82866.335938,0.326452,0.046459
5,_Telephone,77431.695312,0.305042,0.043412
6,_Kitchenette,72257.609375,0.284659,0.040511
7,_Dining_area,70056.367188,0.275987,0.039277
8,_Toaster,61000.109375,0.240310,0.034199
9,_Private_flat_in_building,56850.425781,0.223962,0.031873


#Logistic Regression 

In [26]:
from h2o.estimators.glm import H2OGeneralizedLinearEstimator 
lr = H2OGeneralizedLinearEstimator(family='binomial', alpha=0) 

In [27]:
lr.train(training_frame=train_water, validation_frame=valid_water, x=xc, y='type') 

In [28]:
lr

Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  GLM_model_python_1472160914002_821

GLM Model: summary



,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
,binomial,logit,Ridge ( lambda = 2.369E-4 ),126,126,5,Key_Frame__upload_a12d8959fc47b454fc159b620024a92.hex




ModelMetricsBinomialGLM: glm
** Reported on train data. **

MSE: 0.06158426247521994
RMSE: 0.24816176674745838
LogLoss: 0.20689722888235407
Null degrees of freedom: 209240
Residual degrees of freedom: 209114
Null deviance: 201127.33382892213
Residual deviance: 86582.76613714529
AIC: 86836.76613714529
AUC: 0.9539997055497167
Gini: 0.9079994110994334

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.500456834303554: 


,apartments,hotel,Error,Rate
apartments,28624.0,10329.0,0.2652,(10329.0/38953.0)
hotel,6964.0,163324.0,0.0409,(6964.0/170288.0)
Total,35588.0,173653.0,0.0826,(17293.0/209241.0)



Maximum Metrics: Maximum metrics at their respective thresholds



...

metric,threshold,value,idx
max f1,0.4966347,0.9495390,224.0
max f2,0.2104846,0.9669499,319.0
max f0point5,0.8104532,0.9556340,130.0
max accuracy,0.5109481,0.9173751,220.0
max precision,0.9992271,1.0,0.0
max recall,0.0023380,1.0,398.0
max specificity,0.9992271,1.0,0.0
max absolute_mcc,0.5955119,0.7275130,196.0
max min_per_class_accuracy,0.8445902,0.8894223,115.0
max mean_per_class_accuracy,0.8793088,0.8921972,100.0



Gains/Lift Table: Avg response rate: 81.33 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0100012,0.9991676,1.2295505,1.2295505,1.0,1.0,0.0122969,0.0122969,22.9550453,22.9550453
,2,0.0200023,0.9987174,1.2295505,1.2295505,1.0,1.0,0.0122969,0.0245938,22.9550453,22.9550453
,3,0.0300035,0.9983937,1.2295505,1.2295505,1.0,1.0,0.0122969,0.0368908,22.9550453,22.9550453
,4,0.0400046,0.9980185,1.2248214,1.2283682,0.9961538,0.9990385,0.0122496,0.0491404,22.4821413,22.8368193
,5,0.0500058,0.9977223,1.2248214,1.2276588,0.9961538,0.9984615,0.0122496,0.0613900,22.4821413,22.7658837
,6,0.1000500,0.9963420,1.2286054,1.2281323,0.9992314,0.9988466,0.0614846,0.1228746,22.8605372,22.8132286
,7,0.1499981,0.9948316,1.2229221,1.2263974,0.9946092,0.9974356,0.0610826,0.1839572,22.2922149,22.6397355
,8,0.2000038,0.9933076,1.2196195,1.2247027,0.9919231,0.9960573,0.0609880,0.2449453,21.9619468,22.4702721
,9,0.3000346,0.9907205,1.2269500,1.2254520,0.9978850,0.9966667,0.1227328,0.3676780,22.6949981,22.5451951
,10,0.4000077,0.9852478,1.2224542,1.2247027,0.9942285,0.9960573,0.1222125,0.4898905,22.2454163,22.4702721




Scoring History: 


,timestamp,duration,iteration,negative_log_likelihood,objective
,2016-08-26 08:45:06,0.000 sec,0,100563.6669145,0.4806117
,2016-08-26 08:45:06,0.254 sec,1,51809.8899534,0.2479609
,2016-08-26 08:45:06,0.403 sec,2,44632.1480721,0.2135517
,2016-08-26 08:45:07,0.541 sec,3,43403.9197322,0.2077590
,2016-08-26 08:45:07,0.665 sec,4,43293.4251331,0.2072592
,2016-08-26 08:45:07,0.794 sec,5,43291.3830686,0.2072529


In [29]:
pandas.DataFrame([[k, v] for k, v in lr.coef().items()], columns=['Variable', 'Coefficient'])

,Variable,Coefficient
0,_View_Lake,0.684155
1,_Executive_Lounge_Access,0.229836
2,_Minibar,0.275435
3,_Board_games_puzzles,0.794470
4,_Wardrobe_Closet,0.074059
5,Intercept,1.960500
6,_Toilet,0.575314
7,_Detached,-0.015978
8,_Desk,0.427075
9,_Balcony,-0.537570


#K-means Model 

In [30]:
from h2o.estimators.kmeans import H2OKMeansEstimator 

In [31]:
kmeans = H2OKMeansEstimator(k=2)

In [32]:
kmeans.train(training_frame=train_water, x=xc+['type'])

In [33]:
kmeans

Model Details
H2OKMeansEstimator :  K-means
Model Key:  KMeans_model_python_1472160914002_824

Model Summary: 


,number_of_rows,number_of_clusters,number_of_categorical_columns,number_of_iterations,within_cluster_sum_of_squares,total_sum_of_squares,between_cluster_sum_of_squares
,209241.0,2.0,1.0,15.0,24684709.9603917,26534529.0000000,1849819.0396083




ModelMetricsClustering: kmeans
** Reported on train data. **

MSE: NaN
RMSE: NaN
Total Within Cluster Sum of Square Error: 24684709.96039175
Total Sum of Square Error to Grand Mean: 26534529.000000037
Between Cluster Sum of Square Error: 1849819.0396082886

Centroid Statistics: 


,centroid,size,within_cluster_sum_of_squares
,1.0,34854.0,9952999.5478265
,2.0,174387.0,14731710.4125653



Scoring History: 


,timestamp,duration,iteration,avg_change_of_std_centroids,within_cluster_sum_of_squares
,2016-08-26 08:45:07,0.001 sec,0.0,nan,59163351.1736088
,2016-08-26 08:45:10,2.220 sec,1.0,174.7486033,25143991.3768104
,2016-08-26 08:45:10,2.505 sec,2.0,5.5181024,24741821.2538661
,2016-08-26 08:45:10,2.808 sec,3.0,0.3789257,24702910.6498798
,2016-08-26 08:45:10,3.120 sec,4.0,0.1043115,24690927.7452607
,2016-08-26 08:45:11,3.399 sec,5.0,0.0360588,24686703.0491231
,2016-08-26 08:45:11,3.660 sec,6.0,0.0103538,24685443.1539459
,2016-08-26 08:45:11,3.936 sec,7.0,0.0033607,24685020.5985391
,2016-08-26 08:45:12,4.236 sec,8.0,0.0012296,24684836.3084910
,2016-08-26 08:45:12,4.533 sec,9.0,0.0006555,24684748.9128244


In [34]:
kmeans.centers()

[[0.031416767085556796,
  0.007803982326275299,
  0.30085499512251057,
  0.014259482412348673,
  0.7048258449532332,
  0.9259482412348645,
  0.07129741206174339,
  0.012681471280197353,
  0.29939174843633654,
  0.4560452171917097,
  0.22206920296092414,
  0.6573420554312271,
  0.588626843403915,
  0.31749583978883467,
  0.3156309175417444,
  0.0005738222298731829,
  0.5844953233488299,
  0.6173753371205599,
  0.27945142594824074,
  0.06799793423997284,
  0.5013197911287084,
  0.08062202329718231,
  0.06191541860331706,
  0.12799104837321465,
  0.09622998794973255,
  0.02060021805244732,
  0.03910598496585745,
  0.037499282722212965,
  0.0007746600103287987,
  0.38124748952774445,
  0.0010902622367590527,
  0.8671314626728637,
  0.07723647214093061,
  0.009525449015894867,
  0.23279967865955042,
  0.06590348310093522,
  0.05491478739886346,
  0.5541975096115211,
  0.011246915705514508,
  0.023325873644345185,
  0.004217593389567911,
  0.35094967579044006,
  0.6040052791645188,
  0.03156

In [35]:
centroids = pandas.DataFrame(kmeans.centers_std())

In [36]:
features = train_water.columns.copy()
features.remove('_Size')

In [37]:
centroids['Variable'] = features 
centroids

,0,1,Variable
0,0.191723,-0.0383188,_View_Lake
1,0.00854061,-0.00170698,_Executive_Lounge_Access
2,-0.628109,0.125538,_Telephone
3,0.207322,-0.0414366,_Board_games_puzzles
4,0.754475,-0.150794,_Wardrobe_Closet
5,0.214018,-0.0427749,_Toilet
6,0.425605,-0.0850639,_Detached
7,0.0637899,-0.0127494,_Shared_Toilet
8,-0.448509,0.0896417,_Desk
9,0.678835,-0.135676,_Balcony


In [38]:
lrcoef = pandas.DataFrame([[k, v] for k, v in lr.coef().items()], columns=['Variable', 'Coefficient'])

Coefficients with minus sign correspond to Appartments in Logistic Regression ('Coefficient' column) 

Coefficients with plus sign correspond to Appartments (last column) in K-means 

In [39]:
h = pandas.merge(lrcoef, centroids)

In [63]:
centroids[1].astype(float) - centroids[0].astype(float)

ValueError: could not convert string to float: 'hotel'

-0.024231598466336452
-0.0008596573479226323
0.36800220180156973
-0.013542685862147172
-0.43061389108052456
-0.0918436348135141
-0.06548275844650737
-0.006499817831259117
0.26878133786023284
-0.3194237947232953
-0.21975825088307138
-0.18768548699436022
-0.14119670239569532
0.11099366917685849
-0.11989384998510327
-0.0004304629198328697
-0.5480419179917789
0.09737404213362477
-0.20954942637258467
-0.04539303823281593
0.042086540765298164
-0.06728960746343374
-0.04341059886331308
-0.08603837988301609
-0.08514544609741632
-0.019212499931257246
-0.025314819339979804
-0.03358844074431286
-0.0006485038174933358
-0.07828912680575228
-0.0009124966923090543
-0.7848374843373178
-0.07465027018780376
-0.0029251863816389207
-0.22569479125395572
0.002335548478253416
-0.040544450160423094
-0.36563414192929833
-0.00401587210708105
0.011665266750294535
-0.003988218493503359
-0.29485604495786877
-0.5132875077710015
-0.019609790558064687
-0.27450977401663557
-0.0011822046900408455
-0.3823329492646321
-0.

TypeError: unsupported operand type(s) for -: 'str' and 'str'